In [51]:
import pandas as pd
import requests
from bs4 import BeautifulSoup
from IPython.display import clear_output
import time

In [52]:
paragraphs = pd.DataFrame(columns=['tag', 'text'])
headings = pd.DataFrame(columns=['tag', 'headingtext', 'subheading', 'nextheading', 'paragraphs'])
database = pd.DataFrame(columns=['topicheadingID', 'url', 'totalsubheadings', 'totalparagraphs'])

In [53]:
linkstoscrape = pd.read_csv('hnlinks.csv')
linkstoscrape.head()

,Unnamed: 0,url
0,0,https://www.avax.network/subnets
1,1,/is-blockchain-based-watch-to-earn-monetizatio...
2,2,/pros-and-cons-of-development-on-polygon
3,3,/what-is-stabila-coin-and-how-does-stabila-work
4,4,/doodles-a-brand-exceeding-the-nft-community


In [54]:
scrapedurls = {}
headingdict = {'h1':1, 'h2':2, 'h3':3, 'h4':4, 'h5':5, 'h6':6}

In [55]:
class stack:
    def __init__(self):
        self.__index = []

    def __len__(self):
        return len(self.__index)

    def push(self,item):
        self.__index.insert(0,item)

    def peek(self):
        if len(self) == 0:
            raise Exception("peek() called on empty stack.")
        return self.__index[0]

    def pop(self):
        if len(self) == 0:
            raise Exception("pop() called on empty stack.")
        return self.__index.pop(0)

    def __str__(self):
        return str(self.__index)

def getpage(url):
    page = requests.get(url)
    return BeautifulSoup(page.text, 'html.parser')

In [61]:
i = 0
for urlitem in linkstoscrape['url']:
    if f'https://hackernoon.com{urlitem}' in scrapedurls:
        continue
    if urlitem[0]=='/':
        clear_output(wait=True)
        topic = False
        lastheadingid = 0
        lastheadingnumber = 0
        lastparagraphid = 0
        totalsubheadings = 0
        totalparagraphs = 0
        insideheading = stack()
        url = link
        try:    
            soup = getpage(url)
            main = soup.main
            scrapedurls[url] = True
            descendants = main.descendants
        except:
            continue
            
        for element in descendants:
            try:
                if topic==False:
                    if str(element.name)=='h1' and element.parent.name=='div':
                        docid = len(database.index)
                        topicid = len(headings.index)
#                         database.loc[len(database.index)] = [topicid, url]
                        headings.loc[topicid] = ['topic', element.text, None, None, []]
                        topic = True
                        lastheadingid = topicid
                        lastheadingnumber = 0
                        continue
                    else:
                        continue
                
                if element.name in headingdict:
                    if element.parent.name=='div':
                        totalsubheadings+=1

                        headingid = len(headings.index)

                        headingnumber = headingdict[str(element.name)]
                        headings.loc[headingid] = [element.name, element.text, None, None, []]

                        if lastheadingnumber==0:
                            insideheading.push([lastheadingnumber, lastheadingid])
                            headings.iloc[lastheadingid]['subheading'] = headingid
                        elif headingnumber>lastheadingnumber:
                            insideheading.push([lastheadingnumber, lastheadingid])
                            headings.iloc[lastheadingid]['subheading'] = headingid
                        elif headingnumber==lastheadingnumber:
                            headings.iloc[lastheadingid]['nextheading'] = headingid
                        else:
                            while True:
                                try:
                                    poppedheading = insideheading.pop()
                                    if poppedheading[0]>headingnumber:
                                        continue
                                    elif poppedheading[0]==headingnumber:
                                        headings.iloc[poppedheading[1]]['nextheading'] = headingid
                                        break
                                    else:
                                        insideheading = stack()
                                        headings.iloc[topicid]['subheading'] = headingid
                                        break
                                except:
                                    insideheading = stack()
                                    break
                        lastheadingid = headingid
                        lastheadingnumber = headingnumber
                elif element.name=='p' and element.parent.name=='div':
                    
                    if len(element.text)==0:
                        continue
                    totalparagraphs+=1
                    paragraphid = len(paragraphs.index)
                    paragraphs.loc[paragraphid] = ['p', element.text]
                    headings.iloc[lastheadingid]['paragraphs'].append(paragraphid)
                    lastparagraphid = paragraphid
                elif element.name=='ol' and element.parent.name=='div':
                    
                    totalparagraphs+=1
                    
                    for li in element.find_all('li'):
                        paragraphid = len(paragraphs.index)
                        paragraphs.loc[paragraphid] = ['oli', li.text]
                        headings.iloc[lastheadingid]['paragraphs'].append(paragraphid)
                        lastparagraphid = paragraphid
                elif element.name=='ul' and element.parent.name=='div':
                    
                    totalparagraphs+=1
                    
                    for li in element.find_all('li'):
                        paragraphid = len(paragraphs.index)
                        paragraphs.loc[paragraphid] = ['uli', li.text]
                        headings.iloc[lastheadingid]['paragraphs'].append(paragraphid)
                        lastparagraphid = paragraphid
                elif element.name=='footer':
                    break
                        
            except Exception as e:
                print(element, e)
                break
                
        database.loc[docid] = [topicid, url, totalsubheadings, totalparagraphs]
        print(docid, url)
        if i%25==0 and i!=0:
            j = 0
            for databaseid in database.index:
                if database.iloc[databaseid]['totalsubheadings']+database.iloc[databaseid]['totalparagraphs']!=0:
                    j+=1
            print(j)
            print('sleeping...')
            time.sleep(10)
            i+=1
        else:
            i+=1

7854 https://hackernoon.com/learning-about-financial-sovereignty-and-privacy-with-heliax-founders


# DO NOT TOUCH

In [62]:
headings

,tag,headingtext,subheading,nextheading,paragraphs
0,topic,Is Blockchain-Based Watch-to-Earn Monetization...,2,None,[]
1,h3,@ishantechIshanOnTech,None,None,[0]
2,h2,What Is Happening?,None,3,[1]
3,h2,What is Coub.com?,None,4,"[2, 3]"
4,h2,How is Coubâs Web3 Video Space Going to Work?,None,5,"[4, 5, 6, 7]"
...,...,...,...,...,...
51178,h2,The Graph and GraphQL,None,51179,"[251489, 251490, 251491]"
51179,h2,PARSIQ and ParsiQL,None,51180,"[251492, 251493, 251494, 251495]"
51180,h2,Key differences,None,None,"[251496, 251497, 251498, 251499, 251500, 25150..."
51181,topic,Learning About Financial Sovereignty and Priva...,51182,None,[]


In [63]:
paragraphs

,tag,text
0,p,"Covering the latest events, insights and views..."
1,p,"Coub.com, a micro-video streaming social netwo..."
2,p,Coub is a video-sharing website that is access...
3,p,"On Coub, users can take any video, trim the be..."
4,p,In order to preserve the intrinsic fluidity of...
...,...,...
251530,p,Thank you for answering our questions adrian a...
251531,p,Thanks Sara Pinto and all! I do realise that A...
251532,p,Anoma is a moonshoot. What we are building is ...
251533,p,Check the socials and website here:


In [64]:
database

,topicheadingID,url,totalsubheadings,totalparagraphs
0,0,https://hackernoon.com/is-blockchain-based-wat...,6,13
1,7,https://hackernoon.com/pros-and-cons-of-develo...,5,26
2,13,https://hackernoon.com/what-is-stabila-coin-an...,1,30
3,15,https://hackernoon.com/doodles-a-brand-exceedi...,5,20
4,21,https://hackernoon.com/self-reliance-is-the-on...,1,12
...,...,...,...,...
7850,51145,https://hackernoon.com/what-is-a-lisk-tree-and...,6,48
7851,51152,https://hackernoon.com/the-ultimate-guide-to-n...,14,32
7852,51167,https://hackernoon.com/tech-giants-and-their-b...,6,23
7853,51174,https://hackernoon.com/parsiq-vs-the-graph-pul...,6,29


In [65]:
i = 0
for databaseid in database.index:
    if database.iloc[databaseid]['totalsubheadings']+database.iloc[databaseid]['totalparagraphs']!=0:
        i+=1
print(i)

7817


In [28]:
database.iloc[7838]['url']

'https://hackernoon.com/parsiq-vs-the-graph-pull-and-push-queries-cj1533hi'

In [67]:
database.to_pickle('databasefromhackernoon.pickle')
headings.to_pickle('headingsfromhackernoon.pickle')
paragraphs.to_pickle('paragraphsfromhackernoon.pickle')

In [32]:
soup = getpage('https://hackernoon.com/parsiq-vs-the-graph-pull-and-push-queries-cj1533hi')
for element in soup.main.descendants:
    print(element.name)

div
h1
None
div
div
div
div
div
a
None
None
None
span
i
None
None
None
None
None
button
i
div
div
div
span
img
noscript
img
div
div
label
None
div
div
span
span
img
img
noscript
img
img
img
img
div
span
span
img
img
noscript
img
img
img
img
div
span
span
img
img
noscript
img
img
img
img
div
span
span
img
img
noscript
img
img
img
img
div
div
div
div
span
span
img
img
noscript
img
button
div
div
a
span
img
noscript
img
div
h3
div
a
None
None
None
small
None
p
None
div
a
img
p
None
p
None
a
None
None
p
a
None
None
a
None
None
h2
None
p
None
p
None
p
None
h2
None
p
None
p
None
strong
None
p
None
p
strong
None
None
p
None
a
None
None
p
None
p
None
a
None
None
h2
None
p
None
p
None
p
a
None
None
h2
None
p
None
a
None
None
p
None
div
span
div
div
div
span
span
img
img
noscript
img
button
p
strong
None
p
None
h2
None
p
None
strong
None
p
None
p
strong
None
None
strong
None
None
p
None
a
None
None
a
None
None
a
None
None
a
None
None
a
None
None
a
None
None
p
None
p
None
strong
None
br
br
None
b

In [18]:
url

'https://hackernoon.com/lookup-argument-in-halo2-sin7y-tech-review-14'

In [33]:
urlagain = []
for index in database.index:
    if database.iloc[index]['totalsubheadings']+database.iloc[index]['totalparagraphs']==0:
        urlagain.append(database.iloc[index]['url'])

In [36]:
i = 0
for url in urlagain:
    clear_output(wait=True)
    topic = False
    lastheadingid = 0
    lastheadingnumber = 0
    lastparagraphid = 0
    totalsubheadings = 0
    totalparagraphs = 0
    insideheading = stack()
    try:    
        soup = getpage(url)
        main = soup.main
        scrapedurls[url] = True
        descendants = main.descendants
    except:
        continue

    for element in descendants:
        try:
            if topic==False:
                if str(element.name)=='h1' and element.parent.name=='div':
                    docid = len(database.index)
                    topicid = len(headings.index)
#                         database.loc[len(database.index)] = [topicid, url]
                    headings.loc[topicid] = ['topic', element.text, None, None, []]
                    topic = True
                    lastheadingid = topicid
                    lastheadingnumber = 0
                    continue
                else:
                    continue

            if element.name in headingdict:
                if element.parent.name=='div':
                    totalsubheadings+=1

                    headingid = len(headings.index)

                    headingnumber = headingdict[str(element.name)]
                    headings.loc[headingid] = [element.name, element.text, None, None, []]

                    if lastheadingnumber==0:
                        insideheading.push([lastheadingnumber, lastheadingid])
                        headings.iloc[lastheadingid]['subheading'] = headingid
                    elif headingnumber>lastheadingnumber:
                        insideheading.push([lastheadingnumber, lastheadingid])
                        headings.iloc[lastheadingid]['subheading'] = headingid
                    elif headingnumber==lastheadingnumber:
                        headings.iloc[lastheadingid]['nextheading'] = headingid
                    else:
                        while True:
                            try:
                                poppedheading = insideheading.pop()
                                if poppedheading[0]>headingnumber:
                                    continue
                                elif poppedheading[0]==headingnumber:
                                    headings.iloc[poppedheading[1]]['nextheading'] = headingid
                                    break
                                else:
                                    insideheading = stack()
                                    headings.iloc[topicid]['subheading'] = headingid
                                    break
                            except:
                                insideheading = stack()
                                break
                    lastheadingid = headingid
                    lastheadingnumber = headingnumber
            elif element.name=='p' and element.parent.name=='div':

                if len(element.text)==0:
                    continue
                totalparagraphs+=1
                paragraphid = len(paragraphs.index)
                paragraphs.loc[paragraphid] = ['p', element.text]
                headings.iloc[lastheadingid]['paragraphs'].append(paragraphid)
                lastparagraphid = paragraphid
            elif element.name=='ol' and element.parent.name=='div':

                totalparagraphs+=1

                for li in element.find_all('li'):
                    paragraphid = len(paragraphs.index)
                    paragraphs.loc[paragraphid] = ['oli', li.text]
                    headings.iloc[lastheadingid]['paragraphs'].append(paragraphid)
                    lastparagraphid = paragraphid
            elif element.name=='ul' and element.parent.name=='div':

                totalparagraphs+=1

                for li in element.find_all('li'):
                    paragraphid = len(paragraphs.index)
                    paragraphs.loc[paragraphid] = ['uli', li.text]
                    headings.iloc[lastheadingid]['paragraphs'].append(paragraphid)
                    lastparagraphid = paragraphid
            elif element.name=='footer':
                break

        except Exception as e:
            print(element, e)
            break

    database.loc[docid] = [topicid, url, totalsubheadings, totalparagraphs]
    print(docid, url)
    if i==100:
        break
    else:
        i+=1

7941 https://hackernoon.com/reviewing-the-gitcoin-grants-round-6


In [37]:
database

,topicheadingID,url,totalsubheadings,totalparagraphs
0,0,https://hackernoon.com/is-blockchain-based-wat...,6,13
1,7,https://hackernoon.com/pros-and-cons-of-develo...,5,26
2,13,https://hackernoon.com/what-is-stabila-coin-an...,1,30
3,15,https://hackernoon.com/doodles-a-brand-exceedi...,5,20
4,21,https://hackernoon.com/self-reliance-is-the-on...,1,12
...,...,...,...,...
7937,21995,https://hackernoon.com/sha-256-a-poem,1,11
7938,21997,https://hackernoon.com/bingx-introduces-spot-g...,4,19
7939,22002,https://hackernoon.com/is-proof-of-stake-the-f...,8,32
7940,22011,https://hackernoon.com/talking-crypto-investme...,7,22


In [48]:
1001%100

1